In [0]:
!pip install kaggle
!pip install -U tensorflow-gpu
import os
import numpy as np
from google.colab import files
from keras.optimizers import Nadam
%load_ext tensorboard

In [0]:
files.upload()

In [0]:
import tensorflow as tf
tf.test.is_gpu_available()

In [0]:
!ls

In [0]:
!mkdir /root/.kaggle 
!mv kaggle.json /root/.kaggle
!kaggle competitions download -c dogs-vs-cats


In [0]:
!unzip train.zip
!mkdir -p data/train/cats
!mkdir    data/train/dogs
!mkdir -p data/validation/cats
!mkdir    data/validation/dogs

In [0]:
for i in range(1000):
  os.rename(f"train/cat.{i}.jpg", f"data/train/cats/cat.{i + 1}.jpg")

for i in range(1000, 1400):
  os.rename(f"train/cat.{i}.jpg", f"data/validation/cats/cat.{i + 1 - 1000}.jpg")

In [0]:
for i in range(11100, 12100):
  os.rename(f"train/dog.{i}.jpg", f"data/train/dogs/dog.{i + 1 - 12500}.jpg")
  
for i in range(12100, 12500):
  os.rename(f"train/dog.{i}.jpg", f"data/validation/dogs/dog.{i + 1 - 12100}.jpg")

In [0]:
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Input

In [0]:
img_width, img_height = 150, 150

train_data_dir = 'data/train'
test_data_dir = 'data/validation'

epochs = 50
batch_size = 16
logdir = os.path.join("logs")
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [0]:
inputs = Input(shape=(img_height, img_width, 3))

x = Conv2D(32, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)          


x = Flatten()(x)
latent_vec = Dense(64, activation='relu')(x)
latent_vec = Dropout(0.5)(latent_vec)
output = Dense(1, activation='sigmoid')(latent_vec)

model = Model(inputs, output)


In [0]:
model.summary()

In [0]:
from tensorflow.keras.utils import plot_model

In [0]:
plot_model(model)

In [0]:
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])

In [0]:
train_datagen = ImageDataGenerator(rescale = 1/255)
test_datagen = ImageDataGenerator(rescale = 1/255)

In [0]:
train_generator = train_datagen.flow_from_directory(
                  train_data_dir,
                  target_size=(img_width, img_height),
                  batch_size=batch_size,
                  class_mode='binary'
                  )

test_generator = test_datagen.flow_from_directory(
                  test_data_dir,
                  target_size=(img_width, img_height),
                  batch_size=batch_size,
                  class_mode='binary'
                  )

In [0]:
train_samples = 2000
test_samples = 800

In [0]:
model.fit_generator(train_generator, steps_per_epoch=train_samples//batch_size, 
                    epochs=epochs,
                    callbacks=[tensorboard_callback],
                    validation_data=test_generator,
                    validation_steps=test_samples//batch_size)

In [0]:
%tensorboard --logdir logs

In [0]:
!rm -rf ./logs/ 

In [0]:
train_datagen = ImageDataGenerator(
                rescale = 1/255, 
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True
                )

In [0]:
train_generator = train_datagen.flow_from_directory(
                  train_data_dir,
                  target_size=(img_width, img_height),
                  batch_size=batch_size,
                  class_mode='binary'
                  )

test_generator = test_datagen.flow_from_directory(
                  test_data_dir,
                  target_size=(img_width, img_height),
                  batch_size=batch_size,
                  class_mode='binary'
                  )

In [0]:
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])

In [0]:
model.fit_generator(train_generator, steps_per_epoch=train_samples//batch_size, 
                    epochs=epochs,
                    callbacks=[tensorboard_callback],
                    validation_data=test_generator,
                    validation_steps=test_samples//batch_size)

In [0]:
%tensorboard --logdir logs

In [0]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input

In [0]:
base_model = MobileNet(weights='imagenet', include_top=False, pooling='avg') #imports the mobilenet model and discards the last 1000 neuron layer.
base_model.trainable = False

x = base_model.output
preds = tf.keras.layers.Dense(2,activation='softmax')(x) #final layer with softmax activation

In [0]:
model = Model(inputs=base_model.input,outputs=preds)

In [0]:
model.compile(
  optimizer='nadam',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'])

EPOCHS = 6

model.fit_generator(train_generator, steps_per_epoch=train_samples//batch_size, 
                    epochs=EPOCHS,
                    validation_data=test_generator,
                    validation_steps=test_samples//batch_size)

In [0]:
model.save("mobile.h5")

In [0]:
base_model.trainable = True

In [0]:
model.compile(
  optimizer=Nadam(learning_rate=0.0002),
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'])
model.fit_generator(train_generator, steps_per_epoch=train_samples//batch_size, 
                    epochs=EPOCHS/2,
                    validation_data=test_generator,
                    validation_steps=test_samples//batch_size)